In [23]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import math
import warnings
warnings.filterwarnings("ignore")
# I have used my google drive to load the training and testing data
# Upload the dataset file in your google drive and change the path to run the below line
data= pd.read_csv('/content/drive/MyDrive/ML/Priyadataset2c.csv',dtype='object')
print(data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                 height            diameter             weight  \
0      0.12493777936236    0.13190375787948   0.38616824929558   
1      0.14492137354649    0.12500746617994               0.75   
2     0.071892497830642                0.03    0.1631761874001   
3     0.084260612354435   0.037681879898677   0.23151165834924   
4                  0.07   0.090347950123449   0.19177699658224   
..                  ...                 ...                ...   
115    0.15324808700145     0.1301638325634   0.40118457408591   
116    0.12021522833926    0.11607358210855   0.33882777489585   
117    0.07556917126148   0.072067954099429   0.16160560001853   
118    0.13137129130651   0.091788907990009    0.2781412506351   
119    0.12825404991764    0.14820139358115   0.45533536592389   

                   hue      class  
0      2.9517669794008   Plastic   
1     

Removing the label of dataset for K-means clustering

In [24]:
X_train=data.iloc[:,:-1].values
print(X_train)
Y = data.iloc[:,-1].values
x_train=X_train.astype('float64')
print(Y)

[[' 0.12493777936236' ' 0.13190375787948' ' 0.38616824929558'
  ' 2.9517669794008']
 [' 0.14492137354649' ' 0.12500746617994' ' 0.75' ' 3.4378755151121']
 [' 0.071892497830642' ' 0.03' ' 0.1631761874001' ' 4.0521528606463']
 [' 0.084260612354435' ' 0.037681879898677' ' 0.23151165834924'
  ' 6.2831853071796']
 [' 0.07' ' 0.090347950123449' ' 0.19177699658224' ' 2.1255038511543']
 [' 0.099799766897544' ' 0.10269434067451' ' 0.6361652927456'
  ' 3.5358518383289']
 [' 0.05' ' 0.03' ' 0.1' ' 4.3011769348984']
 [' 0.072371619617917' ' 0.054525468417185' ' 0.12805399057367'
  ' 4.4227553189043']
 [' 0.075784850190286' ' 0.03' ' 0.1' ' 1.4019257428896']
 [' 0.059473195721311' ' 0.066294735104384' ' 0.22178821576542'
  ' 2.3270925629813']
 [' 0.12586422555422' ' 0.1226104246836' ' 0.75' ' 1.315462533672']
 [' 0.16139964853043' ' 0.12626329562681' ' 0.43116589874564'
  ' 2.4909572786528']
 [' 0.074933452440262' ' 0.030261701996066' ' 0.1' ' 3.7327724949663']
 [' 0.08122030230938' ' 0.03021670206

In [25]:
for i in range(len(Y)):
    if Y[i] == ' Plastic ':
        Y[i] = 0
    elif Y[i] == ' Metal ':
        Y[i] = 1
    elif Y[i] == ' Ceramic ':
        Y[i] = 2

3a) Implementing K-Means Clustering

In [26]:
# initializing the centeroids randomly with randomstate = 0
# n_c = number of clusters
def initialize_centroid(X, n_c): 
    rand = np.random.RandomState(0).permutation(X.shape[0]) 
    #print(X[rand[:n_c]])
    return X[rand[:n_c]]

In [27]:
# calculating the euclidean distance to get the distance between the centeroids and the X value
# n_c = numberof clusters
# X = dataset
# c = centeroids
def euclidean_distance(n_c, X, c):
        distance = np.zeros((X.shape[0], n_c))
        X=X.astype('float64')
        c=c.astype('float64')
        #print(type(X))
        for i in range(n_c):
          a = X - c[i, :]
          #print(a)
          r_m = np.linalg.norm(a, axis=1)
          distance[:, i] = np.square(r_m)
        return distance

In [28]:
#Calcluating new centroids by averaging all the points belonging to the centroid
# n_c = number of clusters
# l = labels
def new_centroids(n_c, X, l):
  cent = np.zeros((n_c, X.shape[1]))
  cent = cent.astype('float64')
  X = X.astype('float64')
  for i in range(n_c):
    #print(np.mean(X[l == i, :], axis=0) )
    cent[i, :] = np.mean(X[l == i, :], axis=0) 
  return cent

In [29]:
#  find the minimum distance, so that we can assign the data to a particular cluster
def minimum_distance(distance):
        return np.argmin(distance, axis=1)

In [30]:
# fitting the Kmeans algorithm
def fit_K_means(n_c,X):
        # starting with assigning random centroids
        i_c = initialize_centroid(X,n_c)
        # 100 is the maximum iteration possible
        for i in range(100):
          # assigning the generated centeroid (i_c) and making it as old centeroids (o_c)
            o_c = i_c
            #print(o_c)
            #print(n_c)
            distance = euclidean_distance(n_c,X, o_c)
            l = minimum_distance(distance)
            i_c = new_centroids(n_c,X, l)
            if np.all(o_c == i_c): # checks if they have changed, if not changed break from the loop
                break
        return i_c, l        

In [31]:
# mapping the cluster to classification 
# calculating the accuracy of each cluster and total accuracy
def accuracy(X, c,n_c,Y):
  distance = euclidean_distance(n_c,X, c)
  l = minimum_distance(distance) # get the labels or clusters it belong to
  # initialize each variable
  p_c=0 #count = 0 for plastic class --0
  c_c=0 # count =0 for cermaic class --2
  m_c=0 # count = 0 for metal class --1
  # initializing the correct value to get the total accuracy
  correct=0
  acc =0
  # for each cluster
  for j in range(n_c):
    #print(j)
    l_lst1= np.where(l == j)[0]
    #print(l_lst1)
    for i in (l_lst1):
      if(Y[i] == 0):
        p_c +=1
      elif(Y[i] == 2):
        c_c +=1
      elif(Y[i] == 1):
        m_c +=1
    print("the cluster ",j,"has",p_c,"plastic datapoints")
    print("the cluster ",j,"has",m_c, "metal datapoints")
    print("the cluster ",j,"has",c_c, "ceramic datapoints")
    total = p_c+m_c+c_c
    if(p_c > m_c and p_c > c_c):
      correct += p_c
      print("Since the max is plastic datapoints")
      print("the cluster ",j, " classify it as Plastic")
      print("the cluster ",j," predict plastic with",((p_c)/total)*100," accuracy")
    elif(m_c > p_c and m_c > c_c):
      correct +=m_c
      print("Since the max is metal datapoints")
      print("the cluster ", j, " classify it as Metal")
      print("the cluster ",j," predict metal with",((m_c)/total)*100," accuracy")
    else:
      correct +=c_c
      print("Since the max is ceramic datapoints")
      print("the cluster ", j, " classify it as Ceramic")
      print("the cluster ",j," predict ceramic with ",((c_c)/total)*100," accuracy")
    p_c=0 #0 class
    c_c=0 #2
    m_c=0 #1
  acc = (correct/120) * 100
  print("Total accuracy for kmeans cluster K value ",n_c," is ",acc,"")  

3b) k-Means clustering algorithm to the dataset with K=3, 6, and 9.

Displaying the accuracy for each cluster assuming that the predicted label for each cluster is the most frequently occurring label in this cluster. 

computing the overall accuracy as the weighted sum of the accuracies where the weight is the fraction of data points in the corresponding cluster.


In [32]:
K =[3,6,9]
def find_k(X_train):
    ssd_list = []
    K_list = []
    for k in K:
        print("For K = ",k)
        centroids, labels = fit_K_means(k,X_train) 
        print("Centroids points",centroids)
        print("the cluster labels",labels)
        print(accuracy(X_train,centroids,k,Y))  
        print("------------------------------------------------------------------------------------------")  

In [33]:
find_k(X_train)

For K =  3
Centroids points [[0.09743766 0.08824913 0.33873549 1.65014293]
 [0.10505975 0.09970116 0.42984768 2.7762634 ]
 [0.09551031 0.08315851 0.35992613 4.16007318]]
the cluster labels [1 1 2 2 0 2 2 2 0 1 0 1 2 1 1 2 2 0 1 1 0 1 2 2 1 2 1 2 1 2 2 2 0 1 1 2 1
 0 1 2 1 0 2 1 2 2 2 1 0 1 1 0 2 1 2 2 1 1 1 1 1 1 1 2 1 1 1 0 1 2 2 0 1 1
 1 2 1 0 2 1 2 2 0 2 2 0 2 2 1 1 0 2 1 1 0 2 1 1 2 2 2 2 0 1 2 1 1 2 1 2 2
 2 1 2 2 0 0 1 1 1]
the cluster  0 has 9 plastic datapoints
the cluster  0 has 7 metal datapoints
the cluster  0 has 4 ceramic datapoints
Since the max is plastic datapoints
the cluster  0  classify it as Plastic
the cluster  0  predict plastic with 45.0  accuracy
the cluster  1 has 20 plastic datapoints
the cluster  1 has 13 metal datapoints
the cluster  1 has 19 ceramic datapoints
Since the max is plastic datapoints
the cluster  1  classify it as Plastic
the cluster  1  predict plastic with 38.46153846153847  accuracy
the cluster  2 has 16 plastic datapoints
the cluster  2 has 

After implementing the Kmeans cluatering for k values 3, 6, 9, the maximum accuracy is obtained from k = 9 of 50.833%